In [2]:
import os
import iris
import iris.analysis
import iris.coord_categorisation
import iris.plot
import iris.quickplot
import numpy as np

In [1]:
# ================================================ # Code given to me # ================================================ #
def set_permission(path, mode=0o777):
    '''
    Give a filename(s) permissions

    Inputs:
    filename - a string of the filename
    mode - permission mode (default=0777)n
    '''
    try:
        os.chmod(path, mode)
    except OSError:
        print("Cannot change permissions: {}".format(path))

def save_cube(cube, filename):
    iris.save(cube, filename)
    set_permission(filename)
    print('Saved {}'.format(filename))

def get_analysis_method(stat):
    '''
    returns the iris analysis method from a string  

    Input Arguments:
    ---------------
    * stat
        string of the stat type to use (min, mean, max, sum, stdev or steddev)
    
    Returns:
    -------
    * analysis_method
        iris analysis method for the requested statistic
    '''
    if stat == 'mean':
        analysis_method = iris.analysis.MEAN
    elif stat == 'min':
        analysis_method = iris.analysis.MIN
    elif stat == 'max':
        analysis_method = iris.analysis.MAX
    elif stat == 'sum':
        analysis_method = iris.analysis.SUM
    elif stat in ['stddev', 'stdev']:
        analysis_method = iris.analysis.STD_DEV
    elif stat in ['percentile']:
        analysis_method = iris.analysis.PERCENTILE
    else:
        analysis_method = None
    return analysis_method
 
def add_year_dim(cube):
    '''A function that returns a cube with the year attached as a dimension
 
    Input args:
    ----------
    cube  -- the cube to add the year to
    '''
    try:
        cube.coord('year')
    except:
        iris.coord_categorisation.add_year(cube, 'time')
    return cube

def add_month_dim(cube):
    '''A function that returns a cube with the month attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the month to
    '''
    try:
        cube.remove_coord('month')
    except:
        pass
    iris.coord_categorisation.add_month(cube, 'time', name='month')
    return cube

def add_year_dim(cube):
    '''A function that returns a cube with the year attached as a
    dimension

    Input args:
    ----------
    cube  -- the cube to add the year to
    '''
    try:
        cube.remove_coord('year')
    except:
        pass
    iris.coord_categorisation.add_year(cube, 'time', name='year')
    return cube

def subset_cube_by_season(cube, season):
    '''A function that returns a subset of a cube based on input season
    Input args:
    ----------
    cube  -- the cube to be subset
    mon   -- the month number to subset by
    '''
    try:
        add_season_dim(cube)
    except:
        pass
    subset_cube = cube.extract(iris.Constraint(season=season))
    return subset_cube

def extract_yearly_cubes(cube):
    yearly_cubes = []
    years = set([cell.point.year for cell in cube.coord('time').cells()])
    for year in years:
        year_constraint = iris.Constraint(time=lambda cell: cell.point.year == year)
        year_cube = cube.extract(year_constraint)
        yearly_cubes.append(year_cube)
    iris.cube.CubeList(yearly_cubes)
    return yearly_cubes

def extract_monthly_cubes(cube):
    monthly_cubes = []
    # Extract unique years
    years = set([cell.point.year for cell in cube.coord('time').cells()])
    # Iterate over each year
    for year in sorted(years):  # Sorting to ensure chronological order
        # Iterate over each month
        for month in range(1, 13):
            # Apply constraint for specific year and month
            month_constraint = iris.Constraint(time=lambda cell: cell.point.year == year and cell.point.month == month)
            month_cube = cube.extract(month_constraint)
            if month_cube:  # Check if the cube is not empty
                monthly_cubes.append(month_cube)
    return monthly_cubes

# ================================================================================================ #
# ================================================ # My Code # ================================================ #
def calculate_absolute_vapour_pressure(td_cube):
    """ Calculate absolute vapour pressure array
        Parameters:
            td: dew point temperature (cube object)
        Returns:
            cube object
    """
    filename = (f'{basic_info.data_folder}/es_cube.nc')

    # Check if the file already exists
    if os.path.isfile(filename):
        es_cube = iris.load_cube(filename)
        print('es_cube already exists. Skipping es_cube calculation')
    else:
        print('creating es_cube')
        es_cube = td_cube.copy()
        
        es_cube.data = (0.6108 * np.exp(17.27 * td_cube.data / (237.3 + td_cube.data)))

        es_cube.long_name = "absolute vapour pressure"
        es_cube.var_name = None
        es_cube.units = "kPa"
        save_cube(es_cube, filename)
        with open('basic_info.py', 'a') as f:
            f.write(f"\nes_filepath = '{filename}'")
        print(f'saturation_vapour_pressure_cube (es_cube) saved into the data folder')

    return es_cube

def calculate_saturation_vapour_pressure(tas_cube):
    """ Calculate saturation vapour pressure array
        Parameters:
            tas: temperature at surface (cube object)
        Returns:
            cube object
    """
    filename = (f'{basic_info.data_folder}/saturation_vapour_pressure_cube.nc')
    
    # Check if the file already exists
    if os.path.isfile(filename):
        saturation_vapour_pressure_cube = iris.load_cube(filename)
        print('saturation_vapour_pressure_cube already exists. Skipping saturation_vapour_pressure_cube calculation.')
    else:
        print('creating saturation_vapour_pressure_cube')
        saturation_vapour_pressure_cube = tas_cube.copy()

        saturation_vapour_pressure_cube.data = (0.6108 * np.exp(17.27 * tas_cube.data / (237.3 + tas_cube.data)))
        saturation_vapour_pressure_cube.long_name = "saturation vapour pressure"
        saturation_vapour_pressure_cube.var_name = None
        saturation_vapour_pressure_cube.units = "kPa"

        save_cube(saturation_vapour_pressure_cube, filename)
        with open('basic_info.py', 'a') as f:
            f.write(f"\nsaturation_vapour_pressure_cube_filepath = '{filename}'")
        print(f'saturation_vapour_pressure_cube saved into the data folder')

    return saturation_vapour_pressure_cube

def calculate_relative_humidity(tas_cube, saturation_vapour_pressure_cube):
    """ Calculate relative humidity array
        Parameters:
            tas: temperature at surface (cube object)
            saturation_vapour_pressure: saturation vapour pressure (cube object)
        Returns:
            cube object
    """
    filename = (f'{basic_info.data_folder}/rh_cube.nc')

    # Check if the file already exists
    if os.path.isfile(filename):
        rh_cube = iris.load_cube(filename)   
        print('rh_cube already exists. Skipping rh_cube calculation')
    else:
        print('creating rh_cube')
        rh_cube = tas_cube.copy()
        rh_cube.data = (es_cube.data / saturation_vapour_pressure_cube.data) * 100
        rh_cube = add_year_dim(rh_cube) # Adds a year dimension to the cube
        rh_cube = add_month_dim(rh_cube) # Adds a month dimension to the cube

        rh_cube.long_name = "relative humidity"
        rh_cube.var_name = "RH"
        rh_cube.units = "%"
        
        save_cube(rh_cube, filename)
        with open('basic_info.py', 'a') as f:
            f.write(f"\nrh_filepath = '{filename}'")
        print(f'rh_cube saved into the data folder')

    return rh_cube
# ================================================================================================ #

In [1]:
def create_my_files():
    ''' 
    Creates the following files needed for further analysis:
        tas_cube: temperature at surface (in ˚C)
        rh_cube: relative humidity (in %)
        td_cube: dew point temperature (in ˚C)
        es_cube: water vapor pressure (in kilopascals)
    '''
    global tas_cube, rh_cube, es_cube, td_cube
    # Loading the tas_cube
    if os.path.isfile(filepaths['tas']):
        tas_cube = iris.load_cube(filepaths['tas'])
        print('tas_cube loaded')
        tas_cube.convert_units('celsius')
    else:
        print('Create tas_cube in the Step 2 notebook')

    # Loading the td_cube
    if os.path.isfile(filepaths['td']):
        td_cube = iris.load_cube(filepaths['td'])
        print('td_cube loaded')
        td_cube.convert_units('celsius')
    else:
        print('Create td_cube in the previous notebook')

    # Creating the es_cube 
    es_cube = calculate_absolute_vapour_pressure(td_cube)

    saturation_vapour_pressure_cube = calculate_saturation_vapour_pressure(tas_cube)

    # Creating the rh_cube
    rh_cube = calculate_relative_humidity(tas_cube, saturation_vapour_pressure_cube)

    return tas_cube, rh_cube, es_cube, td_cube